In [ ]:
!pip install -q torch torchvision torchaudio pytorch --upgrade
!pip install -q langchain einops accelerate transformers bitsandbytes scipy
!pip install -q xformers sentencepiece sentence-transformers
!pip install -q llama-index==0.7.21 llama_hub==0.0.19
!pip install -q openai==0.28 pypdf glob2

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

model_name = "meta-llama/Llama-2-7b-chat-hf"

auth_token = "*****************************"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir="./model/", use_auth_token=auth_token
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="./model/",
    use_auth_token=auth_token,
    torch_dtype=torch.float16,
    load_in_8bit=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [5]:
prompt = "### User:Who is Eren yeager? \
          ### Assistant:"

# Prompt to the tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [6]:
len(tokenizer.tokenize(prompt))

15

In [7]:
generated_output = model.generate(
    **inputs, streamer=streamer, use_cache=True, max_new_tokens=float("inf")
)

Eren Yeager is a fictional character and the protagonist of the anime series "Attack on Titan". He is a young man who lives in a world where humans are under attack by giant humanoid creatures known as Titans.

Eren is a skilled fighter and a member of the Scouting Legion, an elite group of soldiers who fight against the Titans. He is also the holder of the Coordinate, a mysterious power that allows him to transform into a Titan and use its abilities in battle.

Throughout the series, Eren struggles with his own identity and the weight of his responsibilities as a hero. He must navigate the dangerous world of the Titans and confront his own demons in order to save humanity and uncover the truth about the world he lives in.

Eren is voiced by Hiroshi Kamiya in the Japanese version of the anime and by Bryce Papenbrook in the English version.

Is there anything else you would like to know about Eren Yeager?


In [66]:
output_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)

### User:Who is Eren yeager?           ### Assistant:Eren Yeager is a fictional character and the protagonist of the anime series "Attack on Titan". He is a young man who lives in a world where humans are under attack by giant humanoid creatures known as Titans.

Eren is a skilled fighter and a member of the Scouting Legion, an elite group of soldiers who fight against the Titans. He is also the holder of the Coordinate, a mysterious power that allows him to transform into a Titan and use its abilities in battle.

Throughout the series, Eren struggles with his own identity and the weight of his responsibilities as a hero. He must navigate the dangerous world of the Titans and confront his own demons in order to save humanity and uncover the truth about the world he lives in.

Eren is voiced by Hiroshi Kamiya in the Japanese version of the anime and by Bryce Papenbrook in the English version.

Is there anything else you would like to know about Eren Yeager?


In [53]:
from llama_index.prompts.prompts import SimpleInputPrompt

# Creating system prompt
system_prompt = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information. <</SYS>>
"""
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

In [55]:
from llama_index.llms import HuggingFaceLLM
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    model=model,
    tokenizer=tokenizer,
)

embeddings = LangchainEmbedding(HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [57]:
from llama_index import set_global_service_context
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    chunk_size=1024, llm=llm, embed_model=embeddings
)

set_global_service_context(service_context)

In [62]:
from llama_index import VectorStoreIndex, download_loader
from pathlib import Path
from glob import glob

PyMuPDFReader = download_loader("PyMuPDFReader")
loader = PyMuPDFReader()


def load_documents(directory):  # Takes the docs directory and returns documents
    documents = []
    for item_path in glob(directory + "*.pdf"):
        documents.extend(loader.load(file_path=Path(item_path), metadata=True))
    return documents


# Load documents
documents = load_documents("./documents/")

In [63]:
len(documents)

7

In [64]:
# Creating an vector index
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

In [68]:
response = query_engine.query(
    "Who are starring in Mahesh babu's latest movie Guntur kaaram?"
)

print(response)

 I'm happy to help! Based on the context information provided, the actors starring in Mahesh Babu's latest movie Guntur Kaaram are:

1. Mahesh Babu
2. Sreeleela
3. Meenakshi Chaudhary
4. Jagapathi Babu
5. Ramya Krishna
6. Jayaram
7. Prakash Raj

Please note that the cast list may not be exhaustive, and there may be additional actors or crew members involved in the film.
